In [13]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd


In [14]:
choose_fl = [
  '../features/cnn_4_aug1_feat.pkl',
 '../features/incept_aug1_feat.pkl',
 '../features/other_feat.pkl',
 '../features/resnet_aug1_feat.pkl',
 '../features/xception_aug1_feat.pkl',

]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

../features/cnn_4_aug1_feat.pkl
../features/incept_aug1_feat.pkl
../features/other_feat.pkl
../features/resnet_aug1_feat.pkl
../features/xception_aug1_feat.pkl
(1604, 15) (8424, 15) (1604,)


In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=1009)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head(20))
        submission.to_csv('../results/less_1800_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission['is_iceberg']=np.clip(test_pred,0.001,0.999)
        print(submission.head(20))
        submission.to_csv('../results/clip_less_1800_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
#         submission = pd.DataFrame()
#         submission['id']=test_df['id']
#         submission['is_iceberg']=w_test_pred
#         print(submission.head())
#         submission.to_csv('../results/weighted_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [16]:
cv_test(3,42,True)

[0]	train-logloss:0.640999	valid-logloss:0.643458
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[105]	train-logloss:0.097947	valid-logloss:0.165277

[0]	train-logloss:0.640639	valid-logloss:0.641882
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[116]	train-logloss:0.089403	valid-logloss:0.170795

[0]	train-logloss:0.639686	valid-logloss:0.639934
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[69]	train-logloss:0.114263	valid-logloss:0.172664

local average valid loss 0.171946785403 val loss std 0.00449227604147
          id  is_iceberg
0   5941774d    0.017403
1   4023181e    0.948221
2   b20200e4    

In [17]:
#cv_test(5,42,True)

In [18]:
#cv_test(10,42,True)